# load file

In [1]:
base_dir = "/home/datamind/can/BASIL/"

In [2]:
import sys
import os
import math
os.chdir(base_dir)
sys.path.append(base_dir)

# BEST2010

In [28]:
import pickle
import pandas as pd
from basil.preprocessing.prep_best_corpus import Best2010NECTEC
from basil.preprocessing.prep_pytorch import PytorchPrepWordLevel

best2010  = Best2010NECTEC()
prep_torch = PytorchPrepWordLevel(best2010.train_data, best2010.val_data, best2010.test_data)
y_true_pos = [prep_torch.pos_to_index[word] for text in prep_torch.pos_test for word in text]
y_true_ner = [prep_torch.ner_to_index[word] for text in prep_torch.ner_test for word in text]
stress_best = pickle.load( open( base_dir+"stress_best2010.p", "rb" ) )
error10 = [item for sublist in stress_best.error10   for item in sublist] 
error20 = [item for sublist in stress_best.error20   for item in sublist] 
error30 = [item for sublist in stress_best.error30   for item in sublist] 
error40 = [item for sublist in stress_best.error40   for item in sublist] 
error50 = [item for sublist in stress_best.error50   for item in sublist] 
error60 = [item for sublist in stress_best.error60   for item in sublist] 
error70 = [item for sublist in stress_best.error70   for item in sublist] 
error80 = [item for sublist in stress_best.error80   for item in sublist] 
error90 = [item for sublist in stress_best.error90   for item in sublist] 
error100 = [item for sublist in stress_best.error100   for item in sublist] 

Preprocessing BEST2010 corpus
Done!


In [29]:


col_names =  ['model','stress level','no_error','last_char','swap_char','move_tone','remove_tone' ]
best_df20  = pd.DataFrame(columns = col_names)
# pos_pred = [item for sublist in pos_pred for item in sublist]
models = ["y_pred_pos_setBiLSTM_Baseline","y_pred_pos_setBiLSTM_ELMO",
        "y_pred_pos_setBiLSTM_U","y_pred_pos_setBiLSTM_U_B","y_pred_pos_setBiLSTM_UBC",
         "y_pred_pos_setBiLSTM_BA","y_pred_pos_setBiLSTM_BA_clp","y_pred_pos_setBiLSTM_U_BCA_clp",
        "y_pred_pos_setBiLSTM_BCAD_clp"]
# models = ["y_pred_pos_setBiLSTM_U"]
for model_name in models:
    pos_pred1 = pickle.load( open( base_dir+"results/"+model_name+"-1.p", "rb" ) )
    pos_pred2 = pickle.load( open( base_dir+"results/"+model_name+"-2.p", "rb" ) )
    pos_pred3 = pickle.load( open( base_dir+"results/"+model_name+"-3.p", "rb" ) )
    matched_pos1 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred1[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos1.append(temp)

    matched_pos2 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred2[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos2.append(temp)

    matched_pos3 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred3[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos3.append(temp)        

    df1 = pd.DataFrame(list(zip(matched_pos1[1],matched_pos1[2],matched_pos1[3],
                               matched_pos1[4],matched_pos1[5],matched_pos1[6],
                               matched_pos1[7],matched_pos1[8],matched_pos1[9],
                               matched_pos1[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 

    df2 = pd.DataFrame(list(zip(matched_pos2[1],matched_pos2[2],matched_pos2[3],
                               matched_pos2[4],matched_pos2[5],matched_pos2[6],
                               matched_pos2[7],matched_pos2[8],matched_pos2[9],
                               matched_pos2[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 
    df3 = pd.DataFrame(list(zip(matched_pos3[1],matched_pos3[2],matched_pos3[3],
                               matched_pos3[4],matched_pos3[5],matched_pos3[6],
                               matched_pos3[7],matched_pos3[8],matched_pos3[9],
                               matched_pos3[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 
    df_rate1 = df1[df1.answer20 == False]['error20'].value_counts()/df1['error20'].value_counts()
#     print(df_rate1)
    df_rate2 = df2[df2.answer20 == False]['error20'].value_counts()/df2['error20'].value_counts()
#     print(df_rate2.sort_index())
    df_rate3 = df3[df3.answer20 == False]['error20'].value_counts()/df3['error20'].value_counts()
#     print(df_rate3.sort_index())
    df_concat = pd.concat((df_rate1,df_rate2,df_rate3)).sort_index()
    avg_result = df_concat.groupby(level=0).mean()
    sd = df_concat.groupby(level=0).std(ddof=1)

    print(model_name)
    my_dic = {'model':model_name,'stress level':20}
    
    for idx,err, s in zip(avg_result.index,avg_result,sd):
        err = float("{0:.2f}".format(err*100))
        s = float("{0:.2f}".format(s*100))
        print(idx,str(err),"+/-",str(s))
        my_dic[idx]=str(err)+" ("+str(s)+")"
    best_df20.loc[len(best_df20)] = my_dic 

y_pred_pos_setBiLSTM_Baseline
last_char 61.84 +/- 0.41
move_tone 63.22 +/- 1.6
no_error 4.94 +/- 0.05
remove_tone 63.23 +/- 2.04
swap_char 57.82 +/- 0.52
y_pred_pos_setBiLSTM_ELMO
last_char 43.28 +/- 0.46
move_tone 46.71 +/- 1.24
no_error 5.22 +/- 0.04
remove_tone 54.1 +/- 1.54
swap_char 53.84 +/- 0.62
y_pred_pos_setBiLSTM_U
last_char 54.95 +/- 0.85
move_tone 46.48 +/- 2.54
no_error 4.6 +/- 0.02
remove_tone 49.1 +/- 0.54
swap_char 47.07 +/- 0.78
y_pred_pos_setBiLSTM_U_B
last_char 53.09 +/- 0.2
move_tone 44.44 +/- 2.78
no_error 4.42 +/- 0.04
remove_tone 49.41 +/- 1.02
swap_char 48.41 +/- 0.6
y_pred_pos_setBiLSTM_UBC
last_char 55.37 +/- 0.22
move_tone 46.64 +/- 3.71
no_error 4.47 +/- 0.07
remove_tone 49.88 +/- 1.69
swap_char 50.27 +/- 0.29
y_pred_pos_setBiLSTM_BA
last_char 55.1 +/- 0.11
move_tone 47.97 +/- 1.56
no_error 4.47 +/- 0.04
remove_tone 52.22 +/- 3.04
swap_char 49.75 +/- 0.24
y_pred_pos_setBiLSTM_BA_clp
last_char 55.32 +/- 0.22
move_tone 50.31 +/- 2.6
no_error 4.4 +/- 0.02
remov

In [30]:

col_names =  ['model','stress level','no_error','last_char','swap_char','move_tone','remove_tone' ]
best_df100  = pd.DataFrame(columns = col_names)

# pos_pred = [item for sublist in pos_pred for item in sublist]
models = ["y_pred_pos_setBiLSTM_Baseline","y_pred_pos_setBiLSTM_ELMO",
        "y_pred_pos_setBiLSTM_U","y_pred_pos_setBiLSTM_U_B","y_pred_pos_setBiLSTM_UBC",
         "y_pred_pos_setBiLSTM_BA","y_pred_pos_setBiLSTM_BA_clp","y_pred_pos_setBiLSTM_U_BCA_clp",
        "y_pred_pos_setBiLSTM_BCAD_clp"]
# models = ["y_pred_pos_setBiLSTM_U"]
for model_name in models:
    pos_pred1 = pickle.load( open( base_dir+"results/"+model_name+"-1.p", "rb" ) )
    pos_pred2 = pickle.load( open( base_dir+"results/"+model_name+"-2.p", "rb" ) )
    pos_pred3 = pickle.load( open( base_dir+"results/"+model_name+"-3.p", "rb" ) )
    matched_pos1 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred1[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos1.append(temp)

    matched_pos2 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred2[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos2.append(temp)

    matched_pos3 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred3[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos3.append(temp)        

    df1 = pd.DataFrame(list(zip(matched_pos1[1],matched_pos1[2],matched_pos1[3],
                               matched_pos1[4],matched_pos1[5],matched_pos1[6],
                               matched_pos1[7],matched_pos1[8],matched_pos1[9],
                               matched_pos1[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 

    df2 = pd.DataFrame(list(zip(matched_pos2[1],matched_pos2[2],matched_pos2[3],
                               matched_pos2[4],matched_pos2[5],matched_pos2[6],
                               matched_pos2[7],matched_pos2[8],matched_pos2[9],
                               matched_pos2[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 
    df3 = pd.DataFrame(list(zip(matched_pos3[1],matched_pos3[2],matched_pos3[3],
                               matched_pos3[4],matched_pos3[5],matched_pos3[6],
                               matched_pos3[7],matched_pos3[8],matched_pos3[9],
                               matched_pos3[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 
    df_rate1 = df1[df1.answer100 == False]['error100'].value_counts()/df1['error100'].value_counts()
#     print(df_rate1)
    df_rate2 = df2[df2.answer100 == False]['error100'].value_counts()/df2['error100'].value_counts()
#     print(df_rate2.sort_index())
    df_rate3 = df3[df3.answer100 == False]['error100'].value_counts()/df3['error100'].value_counts()
#     print(df_rate3.sort_index())
    df_concat = pd.concat((df_rate1,df_rate2,df_rate3)).sort_index()
    avg_result = df_concat.groupby(level=0).mean()
    sd = df_concat.groupby(level=0).std(ddof=1)

    print(model_name)
    my_dic = {'model':model_name,'stress level':100}
    
    for idx,err, s in zip(avg_result.index,avg_result,sd):
        err = float("{0:.2f}".format(err*100))
        s = float("{0:.2f}".format(s*100))
        print(idx,str(err),"+/-",str(s))
        my_dic[idx]=str(err)+" ("+str(s)+")"
    best_df100.loc[len(best_df100)] = my_dic 
    

y_pred_pos_setBiLSTM_Baseline
last_char 77.61 +/- 5.58
move_tone 83.17 +/- 7.16
no_error 5.71 +/- 0.5
remove_tone 82.85 +/- 6.59
swap_char 77.41 +/- 7.89
y_pred_pos_setBiLSTM_ELMO
last_char 47.67 +/- 0.79
move_tone 54.36 +/- 0.46
no_error 4.22 +/- 0.16
remove_tone 60.3 +/- 1.0
swap_char 59.07 +/- 0.04
y_pred_pos_setBiLSTM_U
last_char 73.8 +/- 0.95
move_tone 77.57 +/- 0.99
no_error 4.69 +/- 0.12
remove_tone 76.75 +/- 1.22
swap_char 73.35 +/- 1.14
y_pred_pos_setBiLSTM_U_B
last_char 64.5 +/- 1.21
move_tone 66.54 +/- 2.93
no_error 3.84 +/- 0.07
remove_tone 72.13 +/- 2.6
swap_char 62.6 +/- 1.7
y_pred_pos_setBiLSTM_UBC
last_char 66.02 +/- 0.95
move_tone 66.93 +/- 4.1
no_error 4.19 +/- 0.34
remove_tone 70.55 +/- 4.53
swap_char 62.77 +/- 0.34
y_pred_pos_setBiLSTM_BA
last_char 65.45 +/- 0.75
move_tone 68.72 +/- 1.1
no_error 4.02 +/- 0.43
remove_tone 72.34 +/- 0.84
swap_char 62.08 +/- 0.89
y_pred_pos_setBiLSTM_BA_clp
last_char 67.83 +/- 1.39
move_tone 72.05 +/- 2.64
no_error 4.25 +/- 0.28
remove

In [33]:
df=pd.concat([best_df20,best_df100], axis=0)
df

,model,stress level,no_error,last_char,swap_char,move_tone,remove_tone
0,y_pred_pos_setBiLSTM_Baseline,20,4.94 (0.05),61.84 (0.41),57.82 (0.52),63.22 (1.6),63.23 (2.04)
1,y_pred_pos_setBiLSTM_ELMO,20,5.22 (0.04),43.28 (0.46),53.84 (0.62),46.71 (1.24),54.1 (1.54)
2,y_pred_pos_setBiLSTM_U,20,4.6 (0.02),54.95 (0.85),47.07 (0.78),46.48 (2.54),49.1 (0.54)
3,y_pred_pos_setBiLSTM_U_B,20,4.42 (0.04),53.09 (0.2),48.41 (0.6),44.44 (2.78),49.41 (1.02)
4,y_pred_pos_setBiLSTM_UBC,20,4.47 (0.07),55.37 (0.22),50.27 (0.29),46.64 (3.71),49.88 (1.69)
5,y_pred_pos_setBiLSTM_BA,20,4.47 (0.04),55.1 (0.11),49.75 (0.24),47.97 (1.56),52.22 (3.04)
6,y_pred_pos_setBiLSTM_BA_clp,20,4.4 (0.02),55.32 (0.22),50.5 (0.25),50.31 (2.6),52.62 (0.82)
7,y_pred_pos_setBiLSTM_U_BCA_clp,20,4.41 (0.04),55.07 (0.85),50.14 (0.41),52.66 (0.36),52.93 (0.47)
8,y_pred_pos_setBiLSTM_BCAD_clp,20,4.46 (0.05),56.27 (0.45),50.74 (0.29),51.72 (0.27),53.47 (2.79)
0,y_pred_pos_setBiLSTM_Baseline,100,5.71 (0.5),77.61 (5.58),77.41 (7.89),83.17 (7.16),82.85 (6.59)


In [ ]:
df.to_csv('best_error_ana.csv',index=False)

### ORCHID

In [16]:
from basil.preprocessing.prep_orchid_corpus import ORCHID
from basil.preprocessing.prep_pytorch_ORCHID import PytorchPrepWordLevel

orchid  = ORCHID()
prep_torch = PytorchPrepWordLevel(orchid.train_data, orchid.val_data, orchid.test_data)
y_true_pos = [prep_torch.pos_to_index[word] for text in prep_torch.pos_test for word in text]
stress_orchid = pickle.load( open( base_dir+"stress_orchid.p", "rb" ) )
error10 = [item for sublist in stress_orchid.error10   for item in sublist] 
error20 = [item for sublist in stress_orchid.error20   for item in sublist] 
error30 = [item for sublist in stress_orchid.error30   for item in sublist] 
error40 = [item for sublist in stress_orchid.error40   for item in sublist] 
error50 = [item for sublist in stress_orchid.error50   for item in sublist] 
error60 = [item for sublist in stress_orchid.error60   for item in sublist] 
error70 = [item for sublist in stress_orchid.error70   for item in sublist] 
error80 = [item for sublist in stress_orchid.error80   for item in sublist] 
error90 = [item for sublist in stress_orchid.error90   for item in sublist] 
error100 = [item for sublist in stress_orchid.error100   for item in sublist] 





Preprocessing ORCHID corpus
Done!


In [17]:
stress10 = [item[0] for sublist in stress_orchid.stress10   for item in sublist] 
stress20 = [item[0]  for sublist in stress_orchid.stress20   for item in sublist] 
stress30 = [item[0]  for sublist in stress_orchid.stress30   for item in sublist] 
stress40 = [item[0]  for sublist in stress_orchid.stress40   for item in sublist] 
stress50 = [item[0]  for sublist in stress_orchid.stress50   for item in sublist] 
stress60 = [item[0]  for sublist in stress_orchid.stress60   for item in sublist] 
stress70 = [item[0]  for sublist in stress_orchid.stress70   for item in sublist] 
stress80 = [item[0]  for sublist in stress_orchid.stress80   for item in sublist] 
stress90 = [item[0]  for sublist in stress_orchid.stress90   for item in sublist] 
stress100 = [item[0]  for sublist in stress_orchid.stress100   for item in sublist] 

In [18]:




# pos_pred = [item for sublist in pos_pred for item in sublist]
models = ["y_pred_pos_setBiLSTM_baseline_ORCHID","y_pred_pos_setBiLSTM_ELMO_ORCHID",
        "y_pred_pos_setBiLSTM_U_ORCHID","y_pred_pos_setBiLSTM_UB_ORCHID","y_pred_pos_setBiLSTM_UBC_ORCHID",
         "y_pred_pos_setBiLSTM_UBA_ORCHID","y_pred_pos_setBiLSTM_UBA_clp_ORCHID","y_pred_pos_setBiLSTM_UBCA_clp_ORCHID",
        "y_pred_pos_setBiLSTM_U_BCAD_clp_ORCHID"]
# models = ["y_pred_pos_setBiLSTM_U"]
col_names =  ['model','stress level','no_error','last_char','swap_char','move_tone','remove_tone' ]
orchid_df100  = pd.DataFrame(columns = col_names)
for model_name in models:
    pos_pred1 = pickle.load( open( base_dir+"results/"+model_name+"-1.p", "rb" ) )
    pos_pred2 = pickle.load( open( base_dir+"results/"+model_name+"-2.p", "rb" ) )
    pos_pred3 = pickle.load( open( base_dir+"results/"+model_name+"-3.p", "rb" ) )
    matched_pos1 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred1[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos1.append(temp)

    matched_pos2 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred2[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos2.append(temp)

    matched_pos3 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred3[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos3.append(temp)        

    df1 = pd.DataFrame(list(zip(matched_pos1[1],matched_pos1[2],matched_pos1[3],
                               matched_pos1[4],matched_pos1[5],matched_pos1[6],
                               matched_pos1[7],matched_pos1[8],matched_pos1[9],
                               matched_pos1[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 

    df2 = pd.DataFrame(list(zip(matched_pos2[1],matched_pos2[2],matched_pos2[3],
                               matched_pos2[4],matched_pos2[5],matched_pos2[6],
                               matched_pos2[7],matched_pos2[8],matched_pos2[9],
                               matched_pos2[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 
    df3 = pd.DataFrame(list(zip(matched_pos3[1],matched_pos3[2],matched_pos3[3],
                               matched_pos3[4],matched_pos3[5],matched_pos3[6],
                               matched_pos3[7],matched_pos3[8],matched_pos3[9],
                               matched_pos3[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 
    df_rate1 = df1[df1.answer100 == False]['error100'].value_counts()/df1['error100'].value_counts()
#     print(df_rate1)
    df_rate2 = df2[df2.answer100 == False]['error100'].value_counts()/df2['error100'].value_counts()
#     print(df_rate2.sort_index())
    df_rate3 = df3[df3.answer100 == False]['error100'].value_counts()/df3['error100'].value_counts()
#     print(df_rate3.sort_index())
    df_concat = pd.concat((df_rate1,df_rate2,df_rate3)).sort_index()
    avg_result = df_concat.groupby(level=0).mean()
    sd = df_concat.groupby(level=0).std(ddof=1)
    model_name=model_name.replace("y_pred_pos_set",'')
    model_name=model_name.replace("_ORCHID",'')
    model_name=model_name.replace("_",'-')
    print(model_name)

    my_dic = {'model':model_name,'stress level':100}
    count=0
    for idx,err, s in zip(avg_result.index,avg_result,sd):
        err = float("{0:.2f}".format(err*100))
        s = float("{0:.2f}".format(s*100))
        print(idx,str(err),"+/-",str(s))
        my_dic[idx]=str(err)+" ("+str(s)+")"

    orchid_df100.loc[len(orchid_df100)] = my_dic 

BiLSTM-baseline
last_char 71.3 +/- 2.64
move_tone 62.89 +/- 1.21
no_error 6.49 +/- 1.22
remove_tone 62.6 +/- 3.31
swap_char 69.32 +/- 2.74
BiLSTM-ELMO
last_char 34.67 +/- 0.51
move_tone 41.43 +/- 2.06
no_error 4.16 +/- 0.02
remove_tone 35.67 +/- 1.34
swap_char 51.07 +/- 0.6
BiLSTM-U
last_char 72.23 +/- 1.0
move_tone 65.71 +/- 1.17
no_error 4.78 +/- 0.14
remove_tone 69.42 +/- 2.03
swap_char 70.96 +/- 1.3
BiLSTM-UB
last_char 46.9 +/- 0.8
move_tone 35.32 +/- 2.06
no_error 3.05 +/- 0.14
remove_tone 40.56 +/- 2.52
swap_char 44.14 +/- 1.4
BiLSTM-UBC
last_char 52.21 +/- 1.8
move_tone 40.6 +/- 6.51
no_error 3.33 +/- 0.28
remove_tone 44.08 +/- 2.42
swap_char 50.55 +/- 2.33
BiLSTM-UBA
last_char 57.02 +/- 1.68
move_tone 44.51 +/- 4.24
no_error 3.52 +/- 0.11
remove_tone 47.11 +/- 0.36
swap_char 54.48 +/- 1.28
BiLSTM-UBA-clp
last_char 49.64 +/- 0.87
move_tone 39.37 +/- 9.98
no_error 3.18 +/- 0.2
remove_tone 44.42 +/- 4.47
swap_char 47.17 +/- 1.67
BiLSTM-UBCA-clp
last_char 55.4 +/- 2.77
move_tone 41

In [19]:




# pos_pred = [item for sublist in pos_pred for item in sublist]
models = ["y_pred_pos_setBiLSTM_baseline_ORCHID","y_pred_pos_setBiLSTM_ELMO_ORCHID",
        "y_pred_pos_setBiLSTM_U_ORCHID","y_pred_pos_setBiLSTM_UB_ORCHID","y_pred_pos_setBiLSTM_UBC_ORCHID",
         "y_pred_pos_setBiLSTM_UBA_ORCHID","y_pred_pos_setBiLSTM_UBA_clp_ORCHID","y_pred_pos_setBiLSTM_UBCA_clp_ORCHID",
        "y_pred_pos_setBiLSTM_U_BCAD_clp_ORCHID"]
# models = ["y_pred_pos_setBiLSTM_U"]
col_names =  ['model','stress level','no_error','last_char','swap_char','move_tone','remove_tone' ]
orchid_df20  = pd.DataFrame(columns = col_names)
for model_name in models:
    pos_pred1 = pickle.load( open( base_dir+"results/"+model_name+"-1.p", "rb" ) )
    pos_pred2 = pickle.load( open( base_dir+"results/"+model_name+"-2.p", "rb" ) )
    pos_pred3 = pickle.load( open( base_dir+"results/"+model_name+"-3.p", "rb" ) )
    matched_pos1 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred1[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos1.append(temp)

    matched_pos2 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred2[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos2.append(temp)

    matched_pos3 = [] 
    for i in range(len(pos_pred1)):
        temp=[]
        for word_pred, word_true in zip(pos_pred3[i], y_true_pos):
                temp.append(word_pred==word_true) 
        matched_pos3.append(temp)        

    df1 = pd.DataFrame(list(zip(matched_pos1[1],matched_pos1[2],matched_pos1[3],
                               matched_pos1[4],matched_pos1[5],matched_pos1[6],
                               matched_pos1[7],matched_pos1[8],matched_pos1[9],
                               matched_pos1[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 

    df2 = pd.DataFrame(list(zip(matched_pos2[1],matched_pos2[2],matched_pos2[3],
                               matched_pos2[4],matched_pos2[5],matched_pos2[6],
                               matched_pos2[7],matched_pos2[8],matched_pos2[9],
                               matched_pos2[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 
    df3 = pd.DataFrame(list(zip(matched_pos3[1],matched_pos3[2],matched_pos3[3],
                               matched_pos3[4],matched_pos3[5],matched_pos3[6],
                               matched_pos3[7],matched_pos3[8],matched_pos3[9],
                               matched_pos3[10],error10,error20,error30,error40,
                              error50,error60,error70,error80,error90,error100)), 
                   columns =['answer10','answer20','answer30','answer40','answer50','answer60','answer70','answer80',
                             'answer90','answer100', 'error10','error20','error30','error40',
                            'error50','error60','error70','error80','error90','error100']) 
    df_rate1 = df1[df1.answer20 == False]['error20'].value_counts()/df1['error20'].value_counts()
#     print(df_rate1)
    df_rate2 = df2[df2.answer20 == False]['error20'].value_counts()/df2['error20'].value_counts()
#     print(df_rate2.sort_index())
    df_rate3 = df3[df3.answer20 == False]['error20'].value_counts()/df3['error20'].value_counts()
#     print(df_rate3.sort_index())
    df_concat = pd.concat((df_rate1,df_rate2,df_rate3)).sort_index()
    avg_result = df_concat.groupby(level=0).mean()
    sd = df_concat.groupby(level=0).std(ddof=1)
    model_name=model_name.replace("y_pred_pos_set",'')
    model_name=model_name.replace("_ORCHID",'')
    model_name=model_name.replace("_",'-')
    print(model_name)
    


    my_dic = {'model':model_name,'stress level':20}
    
    for idx,err, s in zip(avg_result.index,avg_result,sd):
        err = float("{0:.2f}".format(err*100))
        s = float("{0:.2f}".format(s*100))
        print(idx,str(err),"+/-",str(s))
        my_dic[idx]=str(err)+" ("+str(s)+")"
    orchid_df20.loc[len(orchid_df20)] = my_dic 

BiLSTM-baseline
last_char 57.98 +/- 0.3
move_tone 48.18 +/- 0.57
no_error 6.34 +/- 0.23
remove_tone 47.25 +/- 2.2
swap_char 59.96 +/- 0.45
BiLSTM-ELMO
last_char 30.28 +/- 0.21
move_tone 38.61 +/- 2.62
no_error 6.76 +/- 0.11
remove_tone 32.97 +/- 2.2
swap_char 49.89 +/- 0.62
BiLSTM-U
last_char 52.07 +/- 1.25
move_tone 55.12 +/- 2.06
no_error 6.51 +/- 0.17
remove_tone 45.79 +/- 1.27
swap_char 52.79 +/- 0.06
BiLSTM-UB
last_char 40.11 +/- 0.69
move_tone 29.7 +/- 2.62
no_error 5.15 +/- 0.11
remove_tone 30.4 +/- 1.27
swap_char 39.97 +/- 2.47
BiLSTM-UBC
last_char 43.07 +/- 0.96
move_tone 33.99 +/- 6.29
no_error 5.13 +/- 0.1
remove_tone 36.63 +/- 4.58
swap_char 43.53 +/- 1.98
BiLSTM-UBA
last_char 49.29 +/- 1.85
move_tone 34.65 +/- 5.24
no_error 5.48 +/- 0.15
remove_tone 39.56 +/- 2.91
swap_char 49.02 +/- 1.08
BiLSTM-UBA-clp
last_char 39.53 +/- 1.05
move_tone 30.03 +/- 6.74
no_error 5.16 +/- 0.1
remove_tone 35.16 +/- 1.1
swap_char 41.39 +/- 2.39
BiLSTM-UBCA-clp
last_char 46.38 +/- 1.31
move_ton

In [20]:
orchid_df20

,model,stress level,no_error,last_char,swap_char,move_tone,remove_tone
0,BiLSTM-baseline,20,6.34 (0.23),57.98 (0.3),59.96 (0.45),48.18 (0.57),47.25 (2.2)
1,BiLSTM-ELMO,20,6.76 (0.11),30.28 (0.21),49.89 (0.62),38.61 (2.62),32.97 (2.2)
2,BiLSTM-U,20,6.51 (0.17),52.07 (1.25),52.79 (0.06),55.12 (2.06),45.79 (1.27)
3,BiLSTM-UB,20,5.15 (0.11),40.11 (0.69),39.97 (2.47),29.7 (2.62),30.4 (1.27)
4,BiLSTM-UBC,20,5.13 (0.1),43.07 (0.96),43.53 (1.98),33.99 (6.29),36.63 (4.58)
5,BiLSTM-UBA,20,5.48 (0.15),49.29 (1.85),49.02 (1.08),34.65 (5.24),39.56 (2.91)
6,BiLSTM-UBA-clp,20,5.16 (0.1),39.53 (1.05),41.39 (2.39),30.03 (6.74),35.16 (1.1)
7,BiLSTM-UBCA-clp,20,5.19 (0.05),46.38 (1.31),47.1 (1.24),36.3 (1.51),39.93 (2.77)
8,BiLSTM-U-BCAD-clp,20,5.17 (0.07),44.39 (2.65),44.56 (2.7),34.65 (3.96),38.46 (2.91)


In [21]:
orchid_df100

,model,stress level,no_error,last_char,swap_char,move_tone,remove_tone
0,BiLSTM-baseline,100,6.49 (1.22),71.3 (2.64),69.32 (2.74),62.89 (1.21),62.6 (3.31)
1,BiLSTM-ELMO,100,4.16 (0.02),34.67 (0.51),51.07 (0.6),41.43 (2.06),35.67 (1.34)
2,BiLSTM-U,100,4.78 (0.14),72.23 (1.0),70.96 (1.3),65.71 (1.17),69.42 (2.03)
3,BiLSTM-UB,100,3.05 (0.14),46.9 (0.8),44.14 (1.4),35.32 (2.06),40.56 (2.52)
4,BiLSTM-UBC,100,3.33 (0.28),52.21 (1.8),50.55 (2.33),40.6 (6.51),44.08 (2.42)
5,BiLSTM-UBA,100,3.52 (0.11),57.02 (1.68),54.48 (1.28),44.51 (4.24),47.11 (0.36)
6,BiLSTM-UBA-clp,100,3.18 (0.2),49.64 (0.87),47.17 (1.67),39.37 (9.98),44.42 (4.47)
7,BiLSTM-UBCA-clp,100,3.27 (0.26),55.4 (2.77),54.26 (2.2),41.02 (2.34),48.42 (0.63)
8,BiLSTM-U-BCAD-clp,100,3.2 (0.18),52.16 (1.38),49.97 (1.35),39.99 (0.95),47.59 (2.03)


In [22]:
df=pd.concat([orchid_df20,orchid_df100])
df

,model,stress level,no_error,last_char,swap_char,move_tone,remove_tone
0,BiLSTM-baseline,20,6.34 (0.23),57.98 (0.3),59.96 (0.45),48.18 (0.57),47.25 (2.2)
1,BiLSTM-ELMO,20,6.76 (0.11),30.28 (0.21),49.89 (0.62),38.61 (2.62),32.97 (2.2)
2,BiLSTM-U,20,6.51 (0.17),52.07 (1.25),52.79 (0.06),55.12 (2.06),45.79 (1.27)
3,BiLSTM-UB,20,5.15 (0.11),40.11 (0.69),39.97 (2.47),29.7 (2.62),30.4 (1.27)
4,BiLSTM-UBC,20,5.13 (0.1),43.07 (0.96),43.53 (1.98),33.99 (6.29),36.63 (4.58)
5,BiLSTM-UBA,20,5.48 (0.15),49.29 (1.85),49.02 (1.08),34.65 (5.24),39.56 (2.91)
6,BiLSTM-UBA-clp,20,5.16 (0.1),39.53 (1.05),41.39 (2.39),30.03 (6.74),35.16 (1.1)
7,BiLSTM-UBCA-clp,20,5.19 (0.05),46.38 (1.31),47.1 (1.24),36.3 (1.51),39.93 (2.77)
8,BiLSTM-U-BCAD-clp,20,5.17 (0.07),44.39 (2.65),44.56 (2.7),34.65 (3.96),38.46 (2.91)
0,BiLSTM-baseline,100,6.49 (1.22),71.3 (2.64),69.32 (2.74),62.89 (1.21),62.6 (3.31)


In [14]:
print(df.to_csv(index=False))


model,stress level,no_error,last_char,swap_char,move_tone,remove_tone
BiLSTM-baseline,20,6.34 (0.23),57.98 (0.3),59.96 (0.45),48.18 (0.57),47.25 (2.2)
BiLSTM-ELMO,20,6.76 (0.11),30.28 (0.21),49.89 (0.62),38.61 (2.62),32.97 (2.2)
BiLSTM-U,20,6.51 (0.17),52.07 (1.25),52.79 (0.06),55.12 (2.06),45.79 (1.27)
BiLSTM-UB,20,5.15 (0.11),40.11 (0.69),39.97 (2.47),29.7 (2.62),30.4 (1.27)
BiLSTM-UBC,20,5.13 (0.1),43.07 (0.96),43.53 (1.98),33.99 (6.29),36.63 (4.58)
BiLSTM-UBA,20,5.16 (0.1),39.53 (1.05),41.39 (2.39),30.03 (6.74),35.16 (1.1)
BiLSTM-UBA-clp,20,5.16 (0.1),39.53 (1.05),41.39 (2.39),30.03 (6.74),35.16 (1.1)
BiLSTM-UBCA-clp,20,5.19 (0.05),46.38 (1.31),47.1 (1.24),36.3 (1.51),39.93 (2.77)
BiLSTM-U-BCAD-clp,20,5.17 (0.07),44.39 (2.65),44.56 (2.7),34.65 (3.96),38.46 (2.91)
BiLSTM-baseline,100,6.49 (1.22),71.3 (2.64),69.32 (2.74),62.89 (1.21),62.6 (3.31)
BiLSTM-ELMO,100,4.16 (0.02),34.67 (0.51),51.07 (0.6),41.43 (2.06),35.67 (1.34)
BiLSTM-U,100,4.78 (0.14),72.23 (1.0),70.96 (1.3),65.71 (1.17)

In [15]:
df.to_csv('orchid_error_ana.csv',index=False)